## Data importation

In [1]:
import dask.dataframe as dd
import pandas as pd

In [2]:
# Read targets
train_events = pd.read_csv('../data/train_events.csv')

In [3]:
train_events.shape

(14508, 5)

In [4]:
train_events.head(10)

,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400
5,038441c925bb,3,wakeup,44400.0,2018-08-17T05:10:00-0400
6,038441c925bb,4,onset,57240.0,2018-08-17T23:00:00-0400
7,038441c925bb,4,wakeup,62856.0,2018-08-18T06:48:00-0400
8,038441c925bb,5,onset,NaN,NaN
9,038441c925bb,5,wakeup,NaN,NaN


**Events data description:**

Sleep logs for series in the training set recording onset and wake events

- series_id - Unique identifier for each series of accelerometer data in train_series.parquet.
- night - An enumeration of potential onset / wakeup event pairs. At most one pair of events can occur for each night.
- event - The type of event, whether onset or wakeup.
- step and timestamp - The recorded time of occurence of the event in the accelerometer series.

In [5]:
# Read test data
test_data = pd.read_parquet('../data/test_series.parquet')

In [6]:
test_data.shape

(450, 5)

In [7]:
test_data.head(10)

,series_id,step,timestamp,anglez,enmo
0,038441c925bb,0,2018-08-14T15:30:00-0400,2.6367,0.0217
1,038441c925bb,1,2018-08-14T15:30:05-0400,2.6368,0.0215
2,038441c925bb,2,2018-08-14T15:30:10-0400,2.6370,0.0216
3,038441c925bb,3,2018-08-14T15:30:15-0400,2.6368,0.0213
4,038441c925bb,4,2018-08-14T15:30:20-0400,2.6368,0.0215
5,038441c925bb,5,2018-08-14T15:30:25-0400,2.6367,0.0217
6,038441c925bb,6,2018-08-14T15:30:30-0400,2.6367,0.0217
7,038441c925bb,7,2018-08-14T15:30:35-0400,2.6367,0.0218
8,038441c925bb,8,2018-08-14T15:30:40-0400,2.7980,0.0223
9,038441c925bb,9,2018-08-14T15:30:45-0400,3.0847,0.0217


As the training data is too large to read directly with Pandas, we will use Dask to import it and to shape it into the desired format.

In [8]:
# Read train data
train_data = dd.read_parquet('../data/train_series.parquet', engine='pyarrow')

In [9]:
train_data

,series_id,step,timestamp,anglez,enmo
npartitions=28,,,,,
,string,uint32,string,float32,float32
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
train_data.head(10)

,series_id,step,timestamp,anglez,enmo
0,038441c925bb,0,2018-08-14T15:30:00-0400,2.6367,0.0217
1,038441c925bb,1,2018-08-14T15:30:05-0400,2.6368,0.0215
2,038441c925bb,2,2018-08-14T15:30:10-0400,2.6370,0.0216
3,038441c925bb,3,2018-08-14T15:30:15-0400,2.6368,0.0213
4,038441c925bb,4,2018-08-14T15:30:20-0400,2.6368,0.0215
5,038441c925bb,5,2018-08-14T15:30:25-0400,2.6367,0.0217
6,038441c925bb,6,2018-08-14T15:30:30-0400,2.6367,0.0217
7,038441c925bb,7,2018-08-14T15:30:35-0400,2.6367,0.0218
8,038441c925bb,8,2018-08-14T15:30:40-0400,2.7980,0.0223
9,038441c925bb,9,2018-08-14T15:30:45-0400,3.0847,0.0217


**Data description:**

Each series is a continuous recording of accelerometer data for a single subject spanning many days.

- series_id - Unique identifier for each accelerometer series.
- step - An integer timestep for each observation within a series.
- timestamp - A corresponding datetime with ISO 8601 format %Y-%m-%dT%H:%M:%S%z.
- anglez - As calculated and described by the GGIR package, z-angle is a metric derived from individual accelerometer components that is commonly used in sleep detection, and refers to the angle of the arm relative to the vertical axis of the body
- enmo - As calculated and described by the GGIR package, ENMO is the Euclidean Norm Minus One of all accelerometer signals, with negative values rounded to zero. While no standard measure of acceleration exists in this space, this is one of the several commonly computed features